In [18]:
import sys
sys.path.insert(0, "..")
import os
from dotenv import load_dotenv
from typing import Optional, List
from azure.core.credentials import AzureSasCredential
#from azure.core.exceptions import ResourceNotFoundError
from azure.storage.blob import ContentSettings, PublicAccess
from azure.storage.blob.aio import BlobServiceClient, ContainerClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
from app.services.azure_store_service import AzureStoreService
from app.services.stt_service import SttService
from uuid import uuid4
import unittest
import asyncio
import nest_asyncio
nest_asyncio.apply() 

from loguru import logger
load_dotenv("../.dev.env")

True

In [19]:
def generate_id():
    return str(uuid4())

In [22]:
user_id = generate_id()
user_id

'e8077c17-c44d-437d-810e-5166c3f2ae97'

In [20]:
account_url = os.getenv("AZURE_ACCOUNT_URL")
#connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
user_id = generate_id()
container_name = "worklog-s2t"

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
chat_deployment = os.getenv("AZURE_GPT_DEPLOYMENT_NAME")
      
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version="2024-02-01",
)

In [30]:
      


prompt = "This is an example of a prompt in English."

response = client.chat.completions.create(
        model=chat_deployment,
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant that helps tram drivers and technicians in their own language.",
            },
            {
                "role": "user",
                "content": f"What is the language of the following text? Answer with just the name of language:\n{prompt}",
            },
        ],
    )

In [31]:
response.choices[0].message.content

'English'

In [8]:
speech_deployment = os.getenv("AZURE_SPEECH_DEPLOYMENT_NAME")
audio_test_file = "../app/data/1_766414.wav"
    
result = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=speech_deployment
    )
    
print(result)

Transcription(text="De melding is van werkoorder 766414 is de deur defect, deur 3. Oké, fotocel zender is vervangen. De melding was fotocel deur 3B blijft geblokkeerd staan. Dat is gelijk niet zo'n heel duidelijk logboek. Lijst, pakken we een andere.")


In [21]:
import requests
import json
from azure.identity import DefaultAzureCredential

token_credential = DefaultAzureCredential()
subscriptionId = os.getenv('AZURE_SUBSCRIPTION_ID') 


url = (
    f"https://management.azure.com/subscriptions/"
    + f"{subscriptionId}/providers/Microsoft.CognitiveServices/locations/"
    + f"{os.getenv('REGION')}/models?api-version={os.getenv('OPENAI_API_VERSION')}"
)
token = token_credential.get_token('https://management.azure.com/.default')
headers = {'Authorization': 'Bearer ' + token.token}

#url = f"https://management.azure.com/subscriptions/{subscriptionId}/providers/Microsoft.CognitiveServices/locations/{region}/models?api-version=2023-05-01"

response = requests.get(url, headers=headers)

data = json.loads(response.text)

print(json.dumps(data, indent=4))

{
    "value": [
        {
            "kind": "OpenAI",
            "skuName": "S0",
            "model": {
                "format": "OpenAI",
                "name": "dall-e-3",
                "version": "3.0",
                "isDefaultVersion": true,
                "skus": [
                    {
                        "name": "Standard",
                        "usageName": "OpenAI.Standard.Dalle",
                        "capacity": {
                            "maximum": 1000,
                            "default": 1
                        },
                        "deprecationDate": "2024-10-31T00:00:00Z",
                        "rateLimits": [
                            {
                                "key": "request",
                                "renewalPeriod": 60,
                                "count": 3
                            }
                        ]
                    }
                ],
                "maxCapacity": 2,
                "capabil

In [13]:
os.getenv('AZURE_SUBSCRIPTION_ID') 
